# ch14 利用SVD简化数据

SVD主要用于奇异值分解，来简化数据，适用于数值型的数据，因为涉及到矩阵运算。

利用SVD，可以得到比原始数据集小得多的数据集，用后者来表示原始数据集进行相关操作，
等同于去除了原始数据集的噪声和冗余信息。可以将SVD看作是从数据集抽取出了相关特征。

SVD应用之一是信息检索，称为隐性语义索引(latent semantic index,LSI)。
> LSI中，文档和词语组成一个矩阵，在该矩阵上应用SVD时，就会构建出多个奇异值，
> 这些奇异值就代表了文档中的概念和主题。

SVD应用之二是推荐系统，先利用SVD从数据中构建出一个主题空间，然后在这个空间下计算相似度。

SVD实际是矩阵分解的一种类型，也就是将原始数据矩阵data分解为三个独立矩阵:$U,\sum,V^T$, 
如果原始数据矩阵data是$m*n$,则矩阵$U,\sum,V^T$ 分别是$m*m,m*n,n*n$。对应如下公式：
$$
Data_{m*n}=U_{m*m}\sum {}_{m*n} V_{n*n}
$$

矩阵$\sum$是一个对角阵，这些元素称为奇异值，它们对应于原始数据集Data的奇异值，等于矩阵
$Data*Data^T$的特征值的平方根。


利用numpy的线性代数工具箱linalg可以调用svd。

两个物品之间相似度的定量方法，有以下几种量化方法：

1. 欧式距离
2. 皮尔逊相关系数，在numpy中是corrcoef()，取值范围是-1到1
3. 余弦相似度，计算的是两个向量夹角的余弦值，如果夹角为90°，则相似度为0.

余弦相似度公式：
$$
cos\theta=\frac{A*B}{||A||*||B||}
$$
其中，||A||表示向量A的2范数，栗子：向量\[4,3,2\]的2范数为$\sqrt{4^2+3^2+2^2}$

三种相似度计算方法对应代码：

In [1]:
import numpy as np 

def eculSim(x,y):
    return 1.0/(1.0+ np.linalg.norm(x-y))

In [2]:
def pearSim(x,y):
    if len(x)<3:
        return 1.0
    # -1到1转换到0到1
    # corr返回四个值，只需要第二个值
    return 0.5 + 0.5*np.corrcoef(x,y,rowvar=0)[0][1] 

In [3]:
def cosSim(x,y):
    num=float(x.T*y)
    denom=np.linalg.norm(x)*np.linalg.norm(y)
    return 0.5+0.5*(num/denom)

In [4]:
# test
def loadExData():
    return[[0, 0, 0, 2, 2],
           [0, 0, 0, 3, 3],
           [0, 0, 0, 1, 1],
           [1, 1, 1, 0, 0],
           [2, 2, 2, 0, 0],
           [5, 5, 5, 0, 0],
           [1, 1, 1, 0, 0]]

In [5]:
data1=np.mat(loadExData())
data=data1
eculSim(data1[:,0],data1[:,4])

0.12973190755680383

In [6]:
print(cosSim(data1[:,0],data1[:,4]),
      cosSim(data1[:,0],data1[:,0]) )

0.5 1.0


In [7]:
print(pearSim(data1[:,0],data1[:,4]),
      pearSim(data1[:,0],data1[:,0]) )

0.20596538173840329 1.0


## 14-5示例：餐馆菜肴推荐系统

描述：假设一个人在家决定外出吃饭，但是并不知到去哪儿，该点什么菜，
那么这个推荐系统可以帮助做到这两点。

推荐系统工作过程是：
+ 寻找用户没有评级的菜
+ 对该用户没有评级的所有物品中，对每个物品预计一个可能的分数，也就是说，假设用户对该物品打分。
+ 对以上这些打分了的物品，按照评分从高到低排序，选择前N个物品。

代码：

In [8]:
# 给定相似度计算方法，计算用户对某一物品的评分值
def stand_estimate(dataArr,user,simMeans,item):
    # item为物品编号
    n=dataArr.shape[1]
    sim_total=0.0
    rate_total=0.0
    # n个物品
    for j in range(n):
        user_rate=dataArr[user,j]
        if user_rate==0.0:
            continue
        # 寻找两个用户都评分的物品
        ids=np.nonzero(np.logical_and(dataArr[:,item].A>0,dataArr[:,j].A>0))[0]
        if len(ids)==0:
            sim_rate=0.0
        else:
            sim_rate=simMeans(dataArr[ids,item],dataArr[ids,j])
        # 相似度加和
        sim_total+=sim_rate
        rate_total+=sim_rate*user_rate
    if sim_total==0:
        return 0
    # 归一化相似度评分，这些评分用于对预测值排序
    return rate_total/sim_total

In [9]:
def recommend(dataMat, user, N=3, simMeas=cosSim, estMethod=stand_estimate):
    unratedItems = np.nonzero(dataMat[user,:].A==0)[1] # find unrated items 
    if len(unratedItems) == 0: return 'you rated everything'
    itemScores = []
    for item in unratedItems:
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        itemScores.append((item, estimatedScore))
    return sorted(itemScores, key=lambda jj: jj[1], reverse=True)[:N]


In [10]:
# test
data1[0,1]=data1[0,0]=data1[1,0]=data1[2,0]=4
data1[3,3]=2
data1

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [5, 5, 5, 0, 0],
        [1, 1, 1, 0, 0]])

In [11]:
recommend(data1,2)

[(2, 2.5), (1, 2.0243290220056256)]

由于用户2，对应矩阵第3列，没有对物品0和1进行评分，所以预测用户2对于物品1和物品2的预测评分，得到预测分分别为2.05和2.5。

接下来，将stand_estimate函数替换为SVD函数。

**基于SVD的评分估计**

利用SVD，将预测得分的矩阵，转换为相对规模小的数据矩阵，换句话说，就是将原始数据集映射到低维空间中去。在低维空间中，利用相似度计算方法，构造出stand_estimate类型的函数作用。

In [12]:
def svd(dataMat, user, simMeas, item):
    n = dataMat.shape[1]
    simTotal = 0.0; ratSimTotal = 0.0
    # SVD分解
    U,Sigma,VT = np.linalg.svd(dataMat)
    Sig4 = np.mat(np.eye(4)*Sigma[:4]) 
    # 取前90的奇异值
    xformedItems=dataMat.T * U[:,:4] * Sig4.I  
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0 or j==item: 
            continue
        similarity = simMeas(xformedItems[item,:].T,\
                             xformedItems[j,:].T)
        print('the %d and %d similarity is: %f' % (item, j, similarity))
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: 
        return 0
    else: 
        return ratSimTotal/simTotal

In [15]:
# test
recommend(data,1,estMethod=svd)

the 1 and 0 similarity is: 0.498142
the 1 and 3 similarity is: 0.498131
the 1 and 4 similarity is: 0.509974
the 2 and 0 similarity is: 0.552670
the 2 and 3 similarity is: 0.552976
the 2 and 4 similarity is: 0.217301


[(2, 3.4177569186592387), (1, 3.3307171545585645)]

In [16]:
data1

matrix([[4, 4, 0, 2, 2],
        [4, 0, 0, 3, 3],
        [4, 0, 0, 1, 1],
        [1, 1, 1, 2, 0],
        [2, 2, 2, 0, 0],
        [5, 5, 5, 0, 0],
        [1, 1, 1, 0, 0]])